## Preamble

I follow GeeksforGeeks to get a basic understanding of each data structures first.

## Utilities

In [2]:
import functools as fn

def printBTree(node, nodeInfo=None, inverted=False, isTop=True):

    # node value string and sub nodes
    stringValue, leftNode, rightNode = nodeInfo(node)

    stringValueWidth  = len(stringValue)

    # recurse to sub nodes to obtain line blocks on left and right
    leftTextBlock     = [] if not leftNode else printBTree(leftNode,nodeInfo,inverted,False)

    rightTextBlock    = [] if not rightNode else printBTree(rightNode,nodeInfo,inverted,False)

    # count common and maximum number of sub node lines
    commonLines       = min(len(leftTextBlock),len(rightTextBlock))
    subLevelLines     = max(len(rightTextBlock),len(leftTextBlock))

    # extend lines on shallower side to get same number of lines on both sides
    leftSubLines      = leftTextBlock  + [""] *  (subLevelLines - len(leftTextBlock))
    rightSubLines     = rightTextBlock + [""] *  (subLevelLines - len(rightTextBlock))

    # compute location of value or link bar for all left and right sub nodes
    #   * left node's value ends at line's width
    #   * right node's value starts after initial spaces
    leftLineWidths    = [ len(line) for line in leftSubLines  ]                            
    rightLineIndents  = [ len(line)-len(line.lstrip(" ")) for line in rightSubLines ]

    # top line value locations, will be used to determine position of current node & link bars
    firstLeftWidth    = (leftLineWidths   + [0])[0]  
    firstRightIndent  = (rightLineIndents + [0])[0] 

    # width of sub node link under node value (i.e. with slashes if any)
    # aims to center link bars under the value if value is wide enough
    # 
    # ValueLine:    v     vv    vvvvvv   vvvvv
    # LinkLine:    / \   /  \    /  \     / \ 
    #
    linkSpacing       = min(stringValueWidth, 2 - stringValueWidth % 2)
    leftLinkBar       = 1 if leftNode  else 0
    rightLinkBar      = 1 if rightNode else 0
    minLinkWidth      = leftLinkBar + linkSpacing + rightLinkBar
    valueOffset       = (stringValueWidth - linkSpacing) // 2

    # find optimal position for right side top node
    #   * must allow room for link bars above and between left and right top nodes
    #   * must not overlap lower level nodes on any given line (allow gap of minSpacing)
    #   * can be offset to the left if lower subNodes of right node 
    #     have no overlap with subNodes of left node                                                                                                                                 
    minSpacing        = 2
    rightNodePosition = fn.reduce(lambda r,i: max(r,i[0] + minSpacing + firstRightIndent - i[1]), \
                                 zip(leftLineWidths,rightLineIndents[0:commonLines]), \
                                 firstLeftWidth + minLinkWidth)

    # extend basic link bars (slashes) with underlines to reach left and right
    # top nodes.  
    #
    #        vvvvv
    #       __/ \__
    #      L       R
    #
    linkExtraWidth    = max(0, rightNodePosition - firstLeftWidth - minLinkWidth )
    rightLinkExtra    = linkExtraWidth // 2
    leftLinkExtra     = linkExtraWidth - rightLinkExtra

    # build value line taking into account left indent and link bar extension (on left side)
    valueIndent       = max(0, firstLeftWidth + leftLinkExtra + leftLinkBar - valueOffset)
    valueLine         = " " * max(0,valueIndent) + stringValue
    slash             = "\\" if inverted else  "/"
    backslash         = "/" if inverted else  "\\"
    uLine             = "¯" if inverted else  "_"

    # build left side of link line
    leftLink          = "" if not leftNode else ( " " * firstLeftWidth + uLine * leftLinkExtra + slash)

    # build right side of link line (includes blank spaces under top node value) 
    rightLinkOffset   = linkSpacing + valueOffset * (1 - leftLinkBar)                      
    rightLink         = "" if not rightNode else ( " " * rightLinkOffset + backslash + uLine * rightLinkExtra )

    # full link line (will be empty if there are no sub nodes)                                                                                                    
    linkLine          = leftLink + rightLink

    # will need to offset left side lines if right side sub nodes extend beyond left margin
    # can happen if left subtree is shorter (in height) than right side subtree                                                
    leftIndentWidth   = max(0,firstRightIndent - rightNodePosition) 
    leftIndent        = " " * leftIndentWidth
    indentedLeftLines = [ (leftIndent if line else "") + line for line in leftSubLines ]

    # compute distance between left and right sublines based on their value position
    # can be negative if leading spaces need to be removed from right side
    mergeOffsets      = [ len(line) for line in indentedLeftLines ]
    mergeOffsets      = [ leftIndentWidth + rightNodePosition - firstRightIndent - w for w in mergeOffsets ]
    mergeOffsets      = [ p if rightSubLines[i] else 0 for i,p in enumerate(mergeOffsets) ]

    # combine left and right lines using computed offsets
    #   * indented left sub lines
    #   * spaces between left and right lines
    #   * right sub line with extra leading blanks removed.
    mergedSubLines    = zip(range(len(mergeOffsets)), mergeOffsets, indentedLeftLines)
    mergedSubLines    = [ (i,p,line + (" " * max(0,p)) )       for i,p,line in mergedSubLines ]
    mergedSubLines    = [ line + rightSubLines[i][max(0,-p):]  for i,p,line in mergedSubLines ]                        

    # Assemble final result combining
    #  * node value string
    #  * link line (if any)
    #  * merged lines from left and right sub trees (if any)
    treeLines = [leftIndent + valueLine] + ( [] if not linkLine else [leftIndent + linkLine] ) + mergedSubLines

    # invert final result if requested
    treeLines = reversed(treeLines) if inverted and isTop else treeLines

    # return intermediate tree lines or print final result
    if isTop : print("\n".join(treeLines))
    else     : return treeLines      

## Binary Tree | Set 1 (Introduction)

In [3]:
from typing import *

# A Python class that represents an individual node in a Binary Tree
class BinaryTreeNode:

    curr_node_value: Any
    left: Optional["BinaryTreeNode"]
    right: Optional["BinaryTreeNode"]

    def __init__(self, curr_node_value):
        self.curr_node_value = curr_node_value
        self.left = None
        self.right = None

    def print_tree(self) -> None:
        """Prints a binary tree.

        Taken from https://stackoverflow.com/questions/48850446/how-to-print-a-binary-tree-in-as-a-structure-of-nodes-in-python
        """
        printBTree(self, lambda n: (str(n.curr_node_value), n.left, n.right))


```python
# create root
root = BinaryTreeNode(1)
''' following is the tree after above statement
        1
      /   \
     None  None'''
 
root.left      = BinaryTreeNode(2);
root.right     = BinaryTreeNode(3);
   
''' 2 and 3 become left and right children of 1
           1
         /   \
        2      3
     /    \    /  \
   None None None None'''
 
root.left.left  = BinaryTreeNode(4);
'''4 becomes left child of 2
           1
       /       \
      2          3
    /   \       /  \
   4    None  None  None
  /  \
None None'''
```

## Binary Tree | Set 2 (Properties)

## Binary Tree | Set 3 (Types of Binary Tree)

## Tree Traversals (Inorder, Preorder and Postorder)

In [4]:
# create a binary tree 1, 2, 3, 4, 5, 6, 7, 8, 9 with left tree 2,3,4,5 and right tree 6,7,8,9
root = BinaryTreeNode(1)
root.left = BinaryTreeNode(2)
root.right = BinaryTreeNode(6)
root.left.left = BinaryTreeNode(3)
root.left.right = BinaryTreeNode(4)
root.left.right.left = BinaryTreeNode(5)
root.right.left = BinaryTreeNode(7)
root.right.right = BinaryTreeNode(9)
root.right.left.left = BinaryTreeNode(8)


In [5]:
root.print_tree()

      1
   __/ \_
  2      6
 / \    / \
3   4  7   9
   /  /
  5  8


### Preorder

Algorithm Preorder(tree):

1. Visit the root.
2. Traverse the left subtree, i.e., call Preorder(left-subtree)
3. Traverse the right subtree, i.e., call Preorder(right-subtree)

#### Recursive [^Recursive Preorder Video Explaination][^Recursive Preorder Article Explaination]

The idea in recursion is in its base case.

In a way, we need to think that every node must see left and see right.


- Define Base Case where when a tree node is `None`, return and exit the function.
- If root is not `None`, then we first visit everything on the left preorder and then we visit everything on the right preorder.
- Start with root node 1 and since it is not `None`, print the value to be $1$.
    - Now at this stage `root` is still our original `root` node so `traverse_preorder_recursive(root.left)` means we visit the left node of $1$ and we see that it is not `None` and is $2$.
    - Now at this stage `root` is now `root.left` (node $2$) and hence when we call `.left` it first check if it is `None`, which it isn't as we have $3$ now.
    - Now at this stage `root` is now `root.left.left` (node $3$) and hence when we call `.left` it first check if it is `None`, which it is and the function terminates without even going to check the right node (why?)
    - But the function terminated at node $3$ and thus the funcion calling `root.left` at node $2$ is left hanging! Therefore, we need to check `.right` of `root.left` which turns out to be $4$ (not `None`). I was very confused why are we visiting right node here, but this is because at each recursive call, the function did not terminate right there, so because at node $2$ we know there is nothing more on the left side since it was already terminated at $3$, we go to the next line `traverse_preorder_recursive(root.right)` and check which we got $4$.
    - Since we are at $4$, we check recursively as well for this node and saw that it has a left node of $5$.
    - And at $5$ we check left again and see it's `None` and terminates and we are back at $4$ again, in which we check its right, this time `None` as well and the recursive children of $2$ are all killed.
    - So we are back to one level before $2$ which is node $1$. And then of course we finally progress to the next line `traverse_preorder_recursive(root.right)` and see the node is $6$.
    - We ask again if left side of the node $6$ is `None` and get $7$.
    - We ask if left side of node $7$ is `None` and we find $8$.
    - We ask if left side of node $8$ is `None` and we find `None` and function terminates.
    - We go back one level up and get $7$ and ask right node and find `None` and function terminates.
    - We go back one level up and get $6$ and ask right node and find $9$.
    - We ask if left side of $9$ has anything and find `None`. Program terminates.


[^Recursive Preorder Video Explaination]: Recursive Preorder Video Explaination: [Pre Order Traversal of Binary Tree (Recursive)](https://www.youtube.com/watch?v=Sg6M9Q-mNXs)
[^Recursive Preorder Article Explaination]: Recursive Preorder Article Explaination: [Preorder Tree Traversal – Iterative and Recursive](https://www.techiedelight.com/preorder-tree-traversal-iterative-recursive/)

In [6]:
# A function to do preorder tree traversal
def traverse_preorder_recursive(root: BinaryTreeNode) -> None:
    """Traverse tree using preorder recursively.

    Args:
        root (BinaryTreeNode): A binary tree node.
    """
    # base case
    if root is None:
        return
    

    # First print the data of node
    print(root.curr_node_value)

    # print("Recurring on left child")
    # Then recur on left child
    traverse_preorder_recursive(root.left)

    # print("Recurring on right child")
    # Finally recur on right child
    traverse_preorder_recursive(root.right)


In [7]:
traverse_preorder_recursive(root)

1
2
3
4
5
6
7
8
9


### Postorder

Algorithm Postorder(tree)
   1. Traverse the left subtree, i.e., call Postorder(left-subtree)
   2. Traverse the right subtree, i.e., call Postorder(right-subtree)
   3. Visit the root.

In [ ]:
# base case: if root.curr_node_value = the main root's value.

In [10]:
# A function to do preorder tree traversal
def traverse_postorder_recursive(root: BinaryTreeNode) -> None:
    """Traverse tree using preorder recursively.

    Args:
        root (BinaryTreeNode): A binary tree node.
    """
    # base case
    if root is None:
        return
    


    # print("Recurring on left child")
    # Then recur on left child
    traverse_postorder_recursive(root.left)

    # print("Recurring on right child")
    # Finally recur on right child
    traverse_postorder_recursive(root.right)
    
    # First print the data of node
    print(root.curr_node_value)

In [12]:
root.print_tree()

      1
   __/ \_
  2      6
 / \    / \
3   4  7   9
   /  /
  5  8


In [11]:
traverse_postorder_recursive(root)

3
5
4
2
8
7
9
6
1


### Inorder Traversal

In [13]:
# A function to do preorder tree traversal
def traverse_inorder_recursive(root: BinaryTreeNode) -> None:
    """Traverse tree using preorder recursively.

    Args:
        root (BinaryTreeNode): A binary tree node.
    """
    # base case
    if root is None:
        return
    


    # print("Recurring on left child")
    # Then recur on left child
    traverse_inorder_recursive(root.left)
    
    # First print the data of node
    print(root.curr_node_value)
    
    # print("Recurring on right child")
    # Finally recur on right child
    traverse_inorder_recursive(root.right)
    


In [14]:
traverse_inorder_recursive(root)

3
2
5
4
1
8
7
6
9
